$\newcommand{\eps}{\epsilon}$

We aim to minimize the value of $C_{9.2}$ in Equation 9.2 of Laczkovich,
$$D_2(s_N (u,x,y); H_f) \leq C_{9.2} N^{-4/3} \ell^{6+6\eps}(N)$$

By Equation 7.2 and 8.6, in the final sentence of the proof of Theorem 9.1, we see that there are two cases: \
Case 1: $\frac{216}{C_{7.1}} E_m \geq 8D_2(S)$ where $0<C_{7.1}<1$ is such that $|\frac{f(x)-f(y)}{x-y}| \geq C_{7.1}$ \
In this case, we have $$C_{9.2} = \frac{216 C_6}{C_{7.1}}$$

Case 2: $\frac{216}{C_{7.1}} E_m < 8D_2(S)$
In this case, we have $C_{9.2} = 8 C_{9.18}$ where $C_{9.18} = c_{ETK}(N+1)C_1 C_2$ is the constant from Wagon 9.18, and $c_{ETK}$ is the constant from Erdos Turan Koksma. Note that $c_{ETK} = \frac{9}{2}$, so $$C_{9.2} = \frac{9}{2}(N+1)C_1 C_2$$


To minimize $C_{9.2}$ as defined in Case 1, we want to minimize $C_6$, which is defined as
$$C_6 = 1 + [C_2 (4C^2 + 512 C + 1) + 4098 C C_1] (\ell (C) + \frac{4}{3})^{6+6\epsilon}$$
where
$$C = \max (\frac{b}{\pi} + b^2 +d, \frac{4}{\sqrt{c}}, 2)$$ from Lemma 9.3, where $b,c,d$ are constants such that $a \leq |f'(x)| \leq b$ and $c \leq |f''(x)| \leq d$;

$$C_1 = \frac{(2 + |\ln{c}|)^{2+2\eps} (\frac{2}{(\ln{2})^{\eps} \eps})^2  (\frac{363}{140 \cdot  2^{1 + \eps}} + \frac{\ln(7)^{-\eps}}{\eps})^2} {1 - \frac{\pi^4}{9} c}$$ from Equations 8.1-8.3 and Wagon 9.16, where $\eps > 0$ and $0 < c <1$; and

$$C_2 = \frac{(2 + |\ln c|)^{2+2\eps} (\frac{2}{(\ln 2)^\eps \eps})^2 (\frac{363}{140 \cdot  2^{1 + \eps}} + \frac{\ln(7)^{-\eps}}{\eps})}{1- \frac{2 \pi^2}{3} c}$$ from Equations 6.1-6.3 and Wagon 9.15, where where $\eps > 0$ and $0 < c <1$ (same ones as $C_1$)

To minimize $C_6$, we want to minimize $C_1$ and $C_2$. We also want $\eps$ to be small, so that $(\ell (C) + \frac{4}{3})^{6+6\epsilon}$ is not too large.

Meanwhile, $C$ is independent of $C_1, C_2$ and should be able to be determined by the exact function $f$. 

In [72]:
import math
import numpy as np
from scipy.optimize import minimize, fsolve, root, root_scalar
from mpmath import nsum, exp, inf

In [2]:
# For wolfram alpha:
# C1: ((2 + |ln(c)|)**(2 + 2*x) * (2 / (ln(2)**x * x))**2 * (363 / (140 * 2**(1+x)) + ln(7)**(-x) / x)**2)/(1 - pi**4 * c / 9)
# C2: ((2 + |ln(c)|)**(2 + 2*x) * (2 / (ln(2)**x * x))**2 * (363 / (140 * 2**(1+x)) + ln(7)**(-x) / x))/(1 - 2*pi**2 * c / 3)
# When plug in c=0.5, eps=x=0.2, wolfram alpha matches with find_C1 and find_C2.
# So the functions should be correct

In [136]:
# For debugging
print_flag = False

In [137]:
ERR_VAL = 10**200

# Constant from Erdos Turan Koksma
etk = 9/2

# Calculate the \ell function
def ell(x):
    if x <= math.e**2: # Note: this is Wagon's defn. Laczkovich uses e instead of e^2
        return 2
    else:
        return math.log(x)

# Calculate the value of C
def find_C(a,b,c,d):
    return max(2, 4/math.sqrt(c), b/math.pi + b**2 +d)

# Default C value for our choice of function f
# C_default = find_C() #TODO: find a,b,c,d... does our choice of function f even work?

# Calculate value of C_1
# Note that we need 0 < c < 9/pi^4 = 0.092, otherwise C_1 will be negative
def find_C1(param):
    c,eps = param
    try:
        numer = (2 + abs(math.log(c)))**(2 + 2*eps) * (2 / (math.log(2)**eps * eps))**2 * (363 / (140 * 2**(1+eps)) + math.log(7)**(-eps) / eps)**2
        denom = 1 - (math.pi)**4 * c / 9
        C1 = numer / denom
    except ZeroDivisionError: 
        C1 = ERR_VAL
    except:
        C1 = ERR_VAL
    if c < 0 or eps < 0 or C1 < 0:
        C1 = ERR_VAL
        
    if print_flag: print("c="+str(c)+", eps="+str(eps)+", C1="+str(C1))
        
    return C1

# Calculate the value of C_2
# Note that we need 0 < c < 3/(2pi^2) = 0.1519, otherwise C_2 will be negative
# Though, if doing this with find_C1, which you prob are, need 0 < c < 9/pi^4 = 0.092
def find_C2(param):
    c,eps = param
    try:
        numer = (2 + abs(math.log(c)))**(2 + 2*eps) * (2 / (math.log(2)**eps * eps))**2 * (363 / (140 * 2**(1+eps)) + math.log(7)**(-eps) / eps)
        denom = 1 - 2*(math.pi)**2 * c / 3 
        C2 = numer/denom 
    except:
        C2 = ERR_VAL
    if c < 0 or eps < 0 or C2 < 0:
        C2 = ERR_VAL
    
    if print_flag: print("c="+str(c)+", eps="+str(eps)+", C2="+str(C2))
        
    return C2
    
# Calculate the value of C_6
def find_C6(param, C): #= C_default):
    c, eps = param
    C1 = find_C1((c, eps))
    C2 = find_C2((c, eps))
    
    if c < 0 or eps < 0 or C1 == ERR_VAL or C2 == ERR_VAL:
        C6 = ERR_VAL
    else:
        C6 = 1 + (C2 * (4 * C**2 + 512 * C + 1) + 4098 * C * C1)*(ell(C) + 4/3)**(6 + 6 * eps)
        if C6 < 0:
            C6 = ERR_VAL
    
    if print_flag: print("c="+str(c)+", eps="+str(eps)+", C6="+str(C6))
        
    return C6

# Calculate the value of C_9.2 in Case 2
def find_C92_case_2(params):
    c, eps, N = params
    return etk*(N+1)*find_C1((c,eps))*find_C2((c,eps))
    

In [138]:
def find_min_C1(init_guess):
    res = minimize(find_C1, init_guess, method='nelder-mead', tol=1e-5)
    if res.success:
        print("The minimum occurs at find_C1"+str(res.x)+" = "+str(res.fun))
    else:
        raise ValueError(res.message)
        
def find_min_C2(init_guess):
    res = minimize(find_C2, init_guess, method='nelder-mead', tol=1e-5)
    if res.success:
        print("The minimum occurs at find_C2"+str(res.x)+" = "+str(res.fun))
    else:
        raise ValueError(res.message)


        
def find_min_C6(init_guess, C): #C is not in init_guess bc C cannot be changed
    res = minimize(find_C6, init_guess, C, method='nelder-mead', tol=1e-5)
    if res.success:
        print("The minimum occurs at find_C6"+str(res.x)+" = "+str(res.fun))
    else:
        raise ValueError(res.message)
    return res.x,float(res.fun)

In [139]:
# Calculate number of pieces based on C6
# K*8*N < 1/2 * N**2 - C_6*N**(2/3)*(ell(N))**(6+6*eps)

def find_K(c, eps, C6):
    K = 1/2 * C6 * nsum(lambda x: 2**(-x/3) * (ell(2**x))**(6+6*eps), [0, inf])
    return float(K)
    #(0.03057295, 0.33633507, 23448803431582.8) gives K=1.42305739900657e+22, 13+8=21 approx 22 so seems good
    
    
# Try to use root-finding methods to find value of N such that inequality holds
# Many don't seem to converge. Try different method.
def helper_func(N, c, eps, C6, K): 
    if N < 0: 
        print("N is negative: N = " + str(N))
        return ERR_VAL
    
    if print_flag: print("N = " +str(N))
    
    result = 1/2*N**2 - C6*N**(2/3)*(ell(N))**(6+6*eps) - 8*K*N
    
    if math.isnan(result):
        print("found nan")
        return ERR_VAL
    
    if print_flag: print("Result = " + str(float(result)))
        
    return float(result)

def find_N(c, eps, C6, K):
    return root_scalar(helper_func, x0 = 2.2774*10**23, x1 = 2.2775*10**23, args=(c, eps, C6, K), 
                       method='secant')
    #return root_scalar(helper_func, bracket=[2.25*10**23,2.3*10**23], args=(c, eps, C6, K), 
                       #method='bisect', xtol = 0.001)

In [144]:
def main():
    C_sixteen = find_C(0.19, 0.75, 2.1, 4) # C_sixteen = 4.801232414637843
    print("C_sixteen = "+str(C_sixteen))
    tup, C6 = find_min_C6([0.03, 0.3], C_sixteen) # c = 0.03057295, eps = 0.33633507, C6 = 23448803431582.8
    c, eps = float(tup[0]), float(tup[1])
    print("c = "+str(c)+", eps = "+str(eps)+", C6 = "+str(C6))
    # c = 0.030572951711381684, eps = 0.33633506779165157, C6 = 23448803431582.8
    K = find_K(c, eps, C6)
    print("K = " + str(K))
    print("Finding value of N: ")
    res_N = find_N(c, eps, C6, K)
    print(res_N)
    print("Found N: " + str(res_N.root))
    print("The number of pieces required for equidecomposition is "+str(64*(res_N.root)**2))

    print(find_C1((c,eps)))
    
    
if __name__ == "__main__":
    main()

C_sixteen = 4.801232414637843
The minimum occurs at find_C6[0.03057295 0.33633507] = 23448803431582.8
c = 0.030572951711381684, eps = 0.33633506779165157, C6 = 23448803431582.8
K = 1.4230573378885336e+22
Finding value of N: 
      converged: True
           flag: 'converged'
 function_calls: 7
     iterations: 6
           root: 2.2774693470657035e+23
Found N: 2.2774693470657035e+23
The number of pieces required for equidecomposition is 3.319594641167284e+48
74149.13258311232
